In [69]:
from textgenrnn import textgenrnn
import pandas
import numpy as np
import rhymes as r
import importlib
importlib.reload(r)
import re

word_intro_model = textgenrnn('weights/intro_model_weights.hdf5',
                                vocab_path='weights/intro_model_vocab.json',
                                config_path='weights/intro_model_config.json')

word_chorus_model = textgenrnn('weights/chorus_model_weights.hdf5',
                                vocab_path='weights/chorus_model_vocab.json',
                                config_path='weights/chorus_model_config.json')

word_verse_model = textgenrnn('weights/verse_model_weights.hdf5',
                                vocab_path='weights/verse_model_vocab.json',
                                config_path='weights/verse_model_config.json')

word_bridge_model = textgenrnn('weights/bridge_model_weights.hdf5',
                                vocab_path='weights/bridge_model_vocab.json',
                                config_path='weights/bridge_model_config.json')

word_outro_model = textgenrnn('weights/outro_model_weights.hdf5',
                                vocab_path='weights/outro_model_vocab.json',
                                config_path='weights/outro_model_config.json')



In [106]:
#uses model to generate a list of lyrics
def gen_lyrics(model):

    lyrics = model.generate(10, temperature=1.2, max_gen_length=1000, return_as_list=True)
    
    return lyrics

#Input: A list of generated lines
#Output: dictionary where the keys are the rhymes and the values are a list of lines that rhyme
def post_process(lines):

    rhyme_dict = dict()

    #Generate of list of last words from each line
    for index, line in enumerate(lines):
        if (len(line) == 0):
            continue
        try:
            wordsInLine = line.split()
            curr_word = ""
            wordsInLine[-1] = wordsInLine[-1].lower()
            #remove puncuation
            for c in wordsInLine[-1]:
                if ord(c) >= 97 and ord(c) <= 122:
                    curr_word += c
            #only add words (remove any empty strings)
            if curr_word != "":
                IT_RHYMES_BABYYY = False
                for key in rhyme_dict.keys():
                    if (r.doTheyRhyme(curr_word, key)):
                        rhyme_dict[key].append(index)
                        IT_RHYMES_BABYYY = True
                if (not IT_RHYMES_BABYYY):
                    rhyme_dict[curr_word] = [index]
        except:
            print (curr_word, 'was not found in the dictionary')
            continue

    return rhyme_dict

#Input dictionary of rhymes with their corresponding line number
#Output: sorts each list of lines in the dictionary
def sort_dict(rhymes_dict):
    
    #loop through each key
    for key in rhymes_dict.keys():
        #get the list of lines associated with the rhyme
        index_list = rhymes_dict[key]
        length = len(index_list)
        if (length < 2):
            continue
        
        #get a list of each distance between the list of lines
        distances = []
        distances.append(index_list[1] - index_list[0])
        for i in range(1, length - 1):
            distances.append(min(index_list[i] - index_list[i - 1], index_list[i + 1] - index_list[i]))

        distances.append(index_list[length - 1] - index_list[length - 2])

        
        #do some cool sorting stuff where you sort distances and the list of lines in parallel
        distances, index_list = zip(*sorted(zip(distances, index_list)))
        distances, index_list = (list(t) for t in zip(*sorted(zip(distances, index_list))))
        print(index_list)
        print(distances)
        
        #add the sorted list back to the dictionary
        rhymes_dict[key] = index_list



#Uses dictionary of rhyming lines to generate lyrics with a user's rhyming sequence
def out1(lines, rhyme_dictionary):
    print ('What rhyming scheme would you like to use? (A to Z)')
    rhyme_scheme = input()
    if (not re.fullmatch('[A-Z]+', rhyme_scheme)):
        print ('That is not a valid rhyming scheme.')
        exit()

    # count the required number of lines for each letter/section of the rhyming scheme
    rhyme_counts = dict()
    for char in rhyme_scheme:
        if char in rhyme_counts:
            rhyme_counts[char] += 1
        else:
            rhyme_counts[char] = 1
    print (rhyme_counts)

    # map each letter/section of the rhyming scheme to a rhyme in rhyme_dict
    used_rhymes = set()
    letter_to_rhyme = dict()
    for letter in rhyme_counts.keys():
        for rhyme in rhyme_dictionary.keys():
            # if the current rhyme has enough rhymes for the current letter in the rhyming scheme
            # AND the rhyme hasn't been used, use it
            if (len(rhyme_dictionary[rhyme]) >= rhyme_counts[letter] and rhyme not in used_rhymes):
                letter_to_rhyme[letter] = rhyme_dictionary[rhyme]
                used_rhymes.add(rhyme)
                break

    print (letter_to_rhyme)

    # print the results!
    rhyme_indices = dict()
    for char in rhyme_scheme:
        if (char not in rhyme_indices):
            print (lines[letter_to_rhyme[char][0]])
            rhyme_indices[char] = 1
        else:
            print (lines[letter_to_rhyme[char][rhyme_indices[char]]])
            rhyme_indices[char] += 1

In [102]:
lines = gen_lyrics(word_verse_model)
lines2 = []
for line in lines:
    split = line.splitlines()
    for l in split:
        lines2.append(l)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.01it/s]


In [107]:
rhymes_dict = post_process(lines2)
sort_dict(rhymes_dict)

mami was not found in the dictionary
lessand was not found in the dictionary
[0, 31]
[31, 31]
[16, 17, 14, 22, 6]
[1, 1, 2, 5, 8]
[7, 36]
[29, 29]
[26, 34]
[8, 8]
[19, 33]
[14, 14]


In [108]:
out1(lines2, rhymes_dict)

What rhyming scheme would you like to use? (A to Z)
AABB
{'A': 2, 'B': 2}
{'A': [0, 31], 'B': [16, 17, 14, 22, 6]}
well a playboy inside neighborhood,, i, turn the key
but don t be like i m just set free
, you, like the, you,
too


In [104]:
for line in lines:
    print(line)

well a playboy inside neighborhood,, i, turn the key
inside how the d turn to how inside but how the d turn inside how d matter
again
(talkin'the chains when i goodbyes
you you to the mess you wanna, him
when it
i m used to to up the run, i don t know what to do

i feel that you might love, it s not just a lie
unless it s that you lighting me up
to me realize that can t love i have my mind

love you, mami, i - i love you, mami
(- i pardon
i the, i, you, i know you (
what i would, you and less,and
never, you,
the
, you, like the, you,
too
all, like
twisted, just, make feel your,
i took us from the bottom

uh in the mirror

boy, would you get up to know you
i know youre always talking
saying you want to stay
every night that you fake you miss me all the that we feel print,

just get up off me of my mind
instead of moving on the
come on, hold,, wont, stop,
i refuse it to go, yeah

come along with me and not scared
but don t be like i m just set free

hello,
how did it was before

nobody k

In [105]:
print(rhymes_dict)

{'key': [0, 31], 'matter': [1], 'again': [2], 'goodbyes': [3], 'him': [4], 'it': [5], 'do': [6, 14, 16, 17, 22], 'lie': [7, 36], 'up': [8], 'mind': [26, 34], 'pardon': [11], 'the': [27], 'like': [18], 'your': [19, 33], 'bottom': [20], 'mirror': [21], 'talking': [23], 'stay': [24], 'print': [25], 'stop': [28], 'yeah': [29], 'scared': [30], 'hello': [32], 'cuffed': [35]}
